In [ ]:
!pip install dwutils@git+ssh://git@gitlab.data.trade.gov.uk/ddatdatascienceteam/data-workspace-utilities.git@latest

In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Massive connected components

Connected components crashes on 90m probabilities. We need to be able to handle that and more.

In [2]:
import cmf
from cmf import clean
from cmf.dedupers import NaiveDeduper
from cmf.helpers import cleaner, cleaners, selector

from dwutils import s3

import pandas as pd
import pyarrow as pa
import rustworkx as rx

ignoring unparsable config /home/jovyan/company-matching/pyproject.toml
ignoring unparsable config /home/jovyan/company-matching/pyproject.toml


In [3]:
with s3.read(path="hmrc_exporters_probabilities.parquet") as f:
    exp_deduped = pd.read_parquet(f, dtype_backend="pyarrow")

exp_deduped.left_id = exp_deduped.left_id.astype("binary[pyarrow]")
exp_deduped.right_id = exp_deduped.right_id.astype("binary[pyarrow]")

exp_deduped.head(3)
exp_deduped.info()

,left_id,right_id,probability
0,b'\\s31\x86\xbb\xd0s\xa2\x92\x8a\xadI< \xc7^+l...,b'v\xa9=\x14\xc2\xc2~\xa7\xbe\xb9\xa2\xe6\xe2M...,1
1,b'?@\xf4\xa9\xbeBQ\xa8\x7fn\xcbT\xac\xedL\x05\...,b'\xf3\xce\xa4\xe4H\r\xcf\xaf\x11IfH\xf9\xc4\x...,1
2,b'\xfe^[\xea\xecLt\x08O\x0b\x11.\xdf*\xcb\x89K...,b'-I\xf4:\xb6\xeb\xb4\xd9\xbb\xe0\xc4\xb7V4\xc...,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90276899 entries, 0 to 90276898
Data columns (total 3 columns):
 #   Column       Dtype          
---  ------       -----          
 0   left_id      binary[pyarrow]
 1   right_id     binary[pyarrow]
 2   probability  int32[pyarrow] 
dtypes: binary[pyarrow](2), int32[pyarrow](1)
memory usage: 4.4 GB


In [17]:
df = exp_deduped.head(10).copy()

In [ ]:
pd.ArrowDtype(pa.Binary)

In [28]:
df[["left_id", "right_id"]].astype("binary[pyarrow]")

,left_id,right_id
0,b'\\s31\x86\xbb\xd0s\xa2\x92\x8a\xadI< \xc7^+l...,b'v\xa9=\x14\xc2\xc2~\xa7\xbe\xb9\xa2\xe6\xe2M...
1,b'?@\xf4\xa9\xbeBQ\xa8\x7fn\xcbT\xac\xedL\x05\...,b'\xf3\xce\xa4\xe4H\r\xcf\xaf\x11IfH\xf9\xc4\x...
2,b'\xfe^[\xea\xecLt\x08O\x0b\x11.\xdf*\xcb\x89K...,b'-I\xf4:\xb6\xeb\xb4\xd9\xbb\xe0\xc4\xb7V4\xc...
3,b'\x07\xab\xa9\x92\xfa`\xc9\x1e\x9b\x12)?w\x9d...,b'\xfc\x81\xdd\xdc\x98V\x96\x88<\x91}\'q\x02y\...
4,b'}C\xd2\xfcxL\x13\xc5\xf4\xe0a\xbdc\x92O.\r\x...,b'\xc7W\xbfPX&\xd2ZIZ\x8a\xac\xae)\xd8\xc9wUQT'
5,b'vP\xff\x91\xfc\t\xd6\xde\xfa\xdc\xc3hj\xe56r...,b'5\xc1\xc4)\xe2G\xbc\xc2\xabr2\xb8\xbc\xc5XH)...
6,b'r\x12H<\x84}/\xbb\xf1`\xa5\xe4\xfc\r\xc8Id\x...,b'Ul\x0b\x8e\x1b^\xb6L\xe3\x8d\xde\x1b\x8d\xdb...
7,b'\xea\x8ab\xc1\xa2r/\x92\xf3\x07\x98\xe6\xcb<...,b'\xe4mK\xffX\xec\xe2\x9e\x92\x1d\xf5\x80\x96p...
8,b'\x999/u#\x1f\x02uw\xcc\x199\x18{\x06\x9b\xc9...,b';&Z.\xed\x16A\xdaP\xfe\xe2{\x7f\xa9\x9d\xb1\...
9,b'Y\x87\xb5M\xe9!Z\x87\xf3\xda\x94e\x0e^p2`\xa...,b'\x84\xfe!s\xd3\xee\xed\xd0\x1a\x81:\x18\x13\...


In [13]:
all_edges = (
    exp_deduped
    .query("probability >= 1")
    .filter(["left_id", "right_id"])
    .itertuples(index=False, name=None)
)

In [14]:
G = rx.PyGraph()
added = {}

for edge in all_edges:
    edge_idx = []
    for sha1 in edge:
        sha1_idx = added.get(sha1)
        if sha1_idx is None:
            sha1_idx = G.add_node(sha1)
            added[sha1] = sha1_idx
        edge_idx.append(sha1_idx)
    edge_idx.append(None)
    _ = G.add_edge(*edge_idx)


In [15]:
rx.number_connected_components(G)

187004

In [19]:
for edge in all_edges:
    print(edge)
    break

b'\\s31\x86\xbb\xd0s\xa2\x92\x8a\xadI< \xc7^+l\xdf'


In [23]:
(
    exp_deduped
    .head(100_000)
    .query("probability >= 1")
    .filter(["left_id", "right_id"])
    .to_records()
)[0]

(0, b'\\s31\x86\xbb\xd0s\xa2\x92\x8a\xadI< \xc7^+l\xdf', b'v\xa9=\x14\xc2\xc2~\xa7\xbe\xb9\xa2\xe6\xe2M\xca\x9d\xf6(\x0b1')